In [3]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
print(f"sklearn version : {sklearn.__version__}")

# just adding a comment
Project = 'spaceship-titanic'

#/kaggle/working
initial_directory = os.getcwd()
online = True if initial_directory=='/kaggle/working' else False
#/kaggle
parrent_directory = os.path.split(initial_directory)[0]
#whether we are working online or locally,os.path.join allows us to deal easily with 

train_file = os.path.join(parrent_directory,'input',Project,"train.csv")
test_file = os.path.join(parrent_directory,'input',Project,"test.csv")
print(train_file,"\n",test_file)

sklearn version : 1.0.2
C:\Users\qarna\Desktop\kaggle\input\spaceship-titanic\train.csv 
 C:\Users\qarna\Desktop\kaggle\input\spaceship-titanic\test.csv


In [20]:
Debugg=False

In [9]:
def add_new_variables(data):
    data['Group'] = data.PassengerId.apply(lambda x : x[0:4])
    data['Group'] = data.groupby('Group')['Group'].transform('count')

    Luxury_variables = ['RoomService','Spa','VRDeck','FoodCourt','ShoppingMall']
    data["Luxury"] = data[Luxury_variables].sum(axis=1)

    data["nbnan"] = data.isnull().sum(axis=1)
    
    data['last_name'] = data['Name'].apply(lambda x:str(x).split(" ")[-1])
    data.last_name.value_counts()
    data['N_Familly_members'] = data.groupby('last_name')['last_name'].transform('count')
    #data['Luxury_Familly'] = data.groupby('last_name')['Luxury'].transform('sum')
    data.set_index('PassengerId')
    
    newvars  = pd.DataFrame(data["Cabin"].str.split('/', expand=True).values,columns=['Cabin_Deck','Cabin_Number','Cabin_Side'])
    data = pd.concat([data,newvars],axis=1)
    return data

In [21]:
data = pd.read_csv(train_file)
data_sub = pd.read_csv(test_file)

if(Debugg):
    data = data.iloc[:30]
    data_sub = data_sub.iloc[:30]
X_sub = data_sub

data = add_new_variables(data)
data_sub = add_new_variables(data_sub)
X_sub = add_new_variables(X_sub)

Target_name = 'Transported'
y = data[Target_name]
X = data.drop(columns=[Target_name])

#"nbnan","ShoppingMall"
columns_to_drop = ["Cabin","Name",'last_name']
X = X.drop(columns=columns_to_drop)
X_sub = X_sub.drop(columns=columns_to_drop)

X = X.drop(columns=['PassengerId'])

numerical_columns = X.select_dtypes(exclude=['object']).columns
categorical_columns = X.select_dtypes(include=['object']).columns

def clean_data(dt):
    #we first use the information we have gathered so far:
    #People from Cabin_Deck==G are al from Europa as shown from : sns.histplot(data=data,x='Cabin_Deck',hue='HomePlanet')
    mask1 = (dt['Cabin_Deck']=='G')
    mask2 = dt.HomePlanet.isnull()
    mask = (mask1) & (mask2)
    dt.loc[mask,'HomePlanet'] = 'Earth' 
    
    mask1 = (dt['HomePlanet']=='Earth')
    mask2 = dt.Cabin_Deck.isnull()
    mask = (mask1) & (mask2)
    dt.loc[mask,'Cabin_Deck'] = 'G' 
    
    # Il n'y a pas de VIP qui ont Earth en HomePlanet : sns.catplot(data=data,x='VIP',col='HomePlanet',kind='count')
    mask1 = (dt['HomePlanet']=='Earth')
    mask2 = dt.VIP.isnull()
    mask = (mask1) & (mask2)
    dt.loc[mask,'VIP'] = False
    
    # aucun des VIP n'est en CryoSleep : sns.histplot(data=X,y='VIP',hue='CryoSleep',multiple='stack')
    mask1 = (dt['CryoSleep']==False)
    mask2 = dt.VIP.isnull()
    mask = (mask1) & (mask2)
    dt.loc[mask,'VIP'] = True 
    
    mask = (dt['VIP']==True)
    mask2 = dt.CryoSleep.isnull()
    mask = (mask1) & (mask2)
    dt.loc[mask,'CryoSleep'] = False
    
    # les gends des decks A B et C sont tous d'europe : sns.histplot(data=data,x='Cabin_Deck',hue='HomePlanet')
    mask1 = (dt.Cabin_Deck.isin(['A','B','C']))
    mask2 = dt.HomePlanet.isnull()
    mask = (mask1) & (mask2)
    dt.loc[mask,'HomePlanet']='Europa'
    
    #Les enfants d'un Age nférieur ou égal à 12 ans n'ont rien dépensé
    mask1 = dt.Age<=12
    for col in ['RoomService','Spa','VRDeck','FoodCourt','ShoppingMall']:
        mask2 = dt[col].isnull()
        mask = (mask1) & (mask2)
        dt.loc[mask,col]= 0
        
    
    Luxury_variables = ['RoomService','Spa','VRDeck','FoodCourt','ShoppingMall']
    dt["Luxury"] = dt[Luxury_variables].sum(axis=1)
    
    dt.loc[(dt.CryoSleep.isnull()) & (dt.Luxury == 0), 'CryoSleep'] = True
    dt.loc[(dt.CryoSleep.isnull()) & (dt.Luxury != 0), 'CryoSleep'] = False
    
    # si l'age est manquant, le selectionner sur la base des habitants de la meme planete    
    dt['Age']=dt['Age'].fillna(dt.groupby('HomePlanet')['Age'].transform("median"))
    
    dt[numerical_columns] = dt[numerical_columns].fillna(dt[numerical_columns].median())
    dt[categorical_columns] = dt[categorical_columns].fillna('Unknown').astype('string')
    return dt

X = clean_data(X)
X_sub = clean_data(X_sub)

In [22]:
from sklearn.base import BaseEstimator, TransformerMixin

def indices_of_top_k(arr, k,reverse=False):
    if(reverse==False):
        z = np.array(arr)[0:k]
    else:
        z = np.array(arr)[-k-1:]
    return np.sort(z)

class TopFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_importances=None, k=None,reverse=False):
        self.feature_importances = feature_importances
        self.k = k
        self.reverse = reverse
    def fit(self, X, y=None):
        if(self.feature_importances!=None):
            self.feature_indices_ = indices_of_top_k(self.feature_importances, self.k,self.reverse)     
                
        return self
    def transform(self, X):
        if(self.feature_importances!=None and self.k!=None):
            return X[:, self.feature_indices_]
        else :
            return X

In [54]:
from sklearn import set_config
set_config(display="diagram")
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


num_preprocessor = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

cat_low_preprocessor = Pipeline([
        #('imputer', SimpleImputer(strategy="most_frequent")),
        ('cat_low_encoder', OneHotEncoder(handle_unknown='ignore')),
    ])

cat_high_preprocessor = Pipeline([
        #('imputer', SimpleImputer(strategy="most_frequent")),
        ('cat_high_encoder', OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)),
    ])


num_columns = X_train.select_dtypes(exclude=['string','object']).columns
cat_columns = X_train.select_dtypes(include='string').columns

cat_low_columns = [col for col in cat_columns if X_train[col].nunique()<10]
cat_high_columns = [col for col in cat_columns if col not in cat_low_columns]

full_preprocessor = ColumnTransformer([
        ("num", num_preprocessor, num_columns),
        ("cat_high", cat_high_preprocessor, cat_high_columns),
        ("cat_low", cat_low_preprocessor, cat_low_columns),
    ],sparse_threshold=0)#remainder="passthrough"
full_preprocessor
    
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
#from xgboost import XGBClassifier
classifier = RandomForestClassifier(n_estimators=400, max_leaf_nodes=10,n_jobs=-1, random_state=42)
#classifier = SGDClassifier(alpha=0.01,random_state=42)
#classifier = HistGradientBoostingClassifier(learning_rate=0.005, max_leaf_nodes=20, early_stopping=True, n_iter_no_change=5, max_iter=1000)

estimators = [
    ("hist",HistGradientBoostingClassifier(learning_rate=0.005, max_leaf_nodes=20, early_stopping=True, n_iter_no_change=5, max_iter=1000,random_state=42)),
    ("forest",RandomForestClassifier(bootstrap=True,n_estimators=600,max_depth=16,max_leaf_nodes=10,n_jobs=-1, random_state=42,min_samples_leaf=1)),
   # ("knn",KNeighborsClassifier(n_neighbors=40)),
    #("sgd",SGDClassifier(alpha=0.01,random_state=42)),
    #("log",LogisticRegression(random_state=42,max_iter=1000)),
   # ("xgb",XGBClassifier(eta=0.005,max_depth=10))
]
classifier = VotingClassifier(estimators, voting='soft')

classifier = SVC(kernel='poly',degree=5,coef0=1,C=1)
classifier = SVC(kernel='rbf',gamma=1,C=10)
#classifier = HistGradientBoostingClassifier(learning_rate=0.005, max_leaf_nodes=20, early_stopping=True, n_iter_no_change=5, max_iter=1000,random_state=42)
#classifier = HistGradientBoostingClassifier(learning_rate=0.004, max_leaf_nodes=20, max_depth = 4,early_stopping=True, n_iter_no_change=5, max_iter=1000,random_state=42)
#classifier = RandomForestClassifier(bootstrap=True,n_estimators=600,max_depth=16,max_leaf_nodes=10,n_jobs=-1, random_state=42,min_samples_leaf=1)
model = Pipeline([
    ("preprocessor", full_preprocessor),
    ("TopFeatureSelector",TopFeatureSelector(feature_importances=None, k=None)),
    ("classifier",  classifier),
])

from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

cv_inner = StratifiedKFold(n_splits=5,shuffle=True,random_state=42) #mega important
cv_outer = StratifiedKFold(n_splits=5,shuffle=True,random_state=42) #mega important
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,shuffle=True)

model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('std_scaler',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Group', 'Luxury', 'nbnan', 'N_Familly_members'],
      dtype='object')),
                                                 ('cat_high',
                                                  Pipeline(steps=[('cat_high_encoder',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1))]),
                                                  ['Cabin_Number']),
                                                 ('cat_low',
                                                  Pipeline(steps=[('cat_low_encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['HomePlanet', 'CryoSleep',
                                                   'Destination', 'VIP',
                                                   'Cabin_Deck',
                                                   'Cabin_Side'])])),
                ('TopFeatureSelector', TopFeatureSelector()),
                ('classifier', SVC(C=10, gamma=1))])

In [55]:
cv_results = cross_validate(model,X_train,y_train,cv=cv_outer,error_score='raise',scoring="balanced_accuracy", return_train_score=True)
for name in ["train", "test"]:
    r = cv_results[f'{name}_score'] 
    print(f"Score {name}: {r.mean():.3f} +/- {r.std():.3f}")
    
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(model,X_train,y_train,cv=cv_outer)
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score
confusion_matrix(y_train,y_train_pred)
print(f"precision_score = {precision_score(y_train,y_train_pred):.3f}")
print(f"recall_score = {recall_score(y_train,y_train_pred):.3f}")
print(f"f1_score = {f1_score(y_train,y_train_pred):.3f}")

Score train: 0.999 +/- 0.000
Score test: 0.603 +/- 0.010
precision_score = 0.659
recall_score = 0.436
f1_score = 0.525
